## Understanding problem statement

- Objective: Create a ML model that optimizes the channel distribution by maximizing market reach and revenue

### Phase 1: EDA
- Understand how variables differ across channels and region
- Visualize on week, city and channel levels
- Insights: Action items to optmize distribution channel

## Ingesting the data

In [8]:
# READING ALL THE DATA IN PANDAS DATAFRAME

import pandas as pd
city_df = pd.read_csv('data/raw/city.csv', encoding='ISO-8859-1')
competitors_df = pd.read_csv('data/raw/competitivelandscape.csv')
consumer_behaviour_df = pd.read_csv('data/raw/consumerbehavior.csv')
market_influencers_df = pd.read_csv('data/raw/externalmarketinfluencers.csv')
products_df = pd.read_csv('data/raw/products.csv')
sales_df = pd.read_csv('data/raw/retailsalesdistribution.csv')

## Univariate 

### Sales data

In [7]:
sales_df.head()

,Date,City_ID,SKU_ID,Channel,Units_Sold,Sales
0,2023-01-01,CT001,SKU1002,Q Commerce,268,80.4
1,2023-01-01,CT001,SKU1004,Q Commerce,168,50.4
2,2023-01-01,CT001,SKU1001,E Commerce,521,156.3
3,2023-01-01,CT001,SKU1002,E Commerce,247,74.1
4,2023-01-01,CT001,SKU1007,E Commerce,161,12075.0


In [9]:
from ydata_profiling import ProfileReport

sales_data_profile = ProfileReport(sales_df, title='Sales Data Profiling Report', explorative=True)


ModuleNotFoundError: No module named 'pkg_resources'